In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
# dataa
df = pd.read_csv("diabetes_modified.csv")  
zero_columns = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
df[zero_columns] = df[zero_columns].replace(0, np.nan)
for col in zero_columns:
    df[col].fillna(df[col].median(), inplace=True)


C:\Users\soumi\AppData\Local\Temp\ipykernel_38104\3486938660.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\soumi\AppData\Local\Temp\ipykernel_38104\3486938660.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [3]:
X = df.drop(columns=["Outcome"])
y = df["Outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [4]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long) 
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [5]:
in_f = 8
fl = 64
sl = 32
ol = 3 # ( no, 1, 2 )

model = nn.Sequential(
    nn.Linear(in_f, fl),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(fl, sl),
    nn.ReLU(),
    nn.Linear(sl, ol),  # 3 output classes
    nn.Softmax(dim=1)  # Softmax for multi-class classification
)


In [6]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [15]:
epochs = 50
for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)  # No need to reshape batch_y
        loss.backward()
        optimizer.step()
    print(loss.item())

0.7184019684791565
0.7181112766265869
0.7259840965270996
0.7181112766265869
0.5514448285102844
0.7248196601867676
0.9781561493873596
0.5514447093009949
0.7181113362312317
0.751053512096405
0.596653163433075
0.5514451265335083
0.5514445900917053
0.8847638964653015
0.5514573454856873
0.5575502514839172
0.5514456629753113
0.7181113362312317
0.9279866218566895
0.8847779631614685
0.8847779631614685
0.7181033492088318
0.718292236328125
0.5514445900917053
0.8847779631614685
0.5514445900917053
0.8847792744636536
0.5514447093009949
0.5514447093009949
0.7181113362312317
0.8415515422821045
0.8794471621513367
0.7181113362312317
0.5514450073242188
0.5514446496963501
0.5514445900917053
0.8931140303611755
0.7182843685150146
0.8848175406455994
0.7183659076690674
1.05144464969635
0.7181113362312317
0.7181113362312317
0.7318820953369141
0.8831830024719238
0.7181155681610107
0.8847780227661133
0.718130350112915
1.0501646995544434
0.7181265950202942


In [16]:
with torch.no_grad():
    y_pred_prob = model(X_test_tensor)
    y_pred = torch.argmax(y_pred_prob, dim=1)  



accuracy = (y_pred.eq(y_test_tensor).sum().item()) / y_test_tensor.shape[0]
print(f"test accuracy: {accuracy:.4f}")

test accuracy: 0.9234
